In [2]:
!pip install transformers

  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached tokenizers-0.12.1-cp36-cp36m-win_amd64.whl
  Using cached regex-2022.10.31-cp36-cp36m-win_amd64.whl (279 kB)
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 16.8
    Uninstalling packaging-16.8:
      Successfully uninstalled packaging-16.8


In [35]:
import pandas as pd
import numpy as np
import glob
import os
# import matplotlib.pyplot as plt
# import seaborn as sns
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from itertools import islice
#from transformers import AutoTokenizer
from scipy.special import softmax
from sklearn.metrics import classification_report
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm
import os
import nltk
from nltk.corpus import stopwords

In [40]:
pd.set_option('display.max_colwidth',None)

## Read Data

In [4]:
ha_df_path = r'semeval2023-afrisenti-main/data/raw/train/ha_train.tsv'
ha_df = pd.read_csv(ha_df_path, index_col=None, sep='\t', header=0)

In [5]:
ha_df.head()

,ID,tweet,label
0,ha_train_00001,@user Da kudin da Arewa babu wani abin azo aga...,negative
1,ha_train_00002,@user Kaga wani Adu ar Banda💔😭 wai a haka Shi ...,negative
2,ha_train_00003,@user Sai haquri fa yan madrid daman kunce cha...,negative
3,ha_train_00004,@user Hmmm yanzu kai kasan girman allah daxaka...,negative
4,ha_train_00005,@user @user Wai gwamno nin Nigeria suna afa kw...,negative


## Preprocessing

In [6]:
# FUNCTIONS TO REMOVE EMOJIS / PUNCTUATION CHARACTERS / WORDS WITH LENGTH LESS THAN 3 / BANNED WORDS:

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


def remove_puncs(item):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~1234567890‹“…‘—’”'''
    for char in item:
        if char in punc:
            item = item.replace(char, "")
    return item

#re.sub(r'\b\w{1,3}\b', '', c)

def remove_less_three_char(item):
    if len(item) >= 3:
        return item
    else:
        return ""

banned = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 
          'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most',
          'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until',
          'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 
          'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 
          'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 
          'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself',
          'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 
          'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than', 'user']
# banned = ['@user', '@user:'] # add stopped words here

In [7]:
## Preprocessing 

def preprocessing(df):

    df_translated_combined_new = df #df_translated_combined #df_raw_combined

    #Create list
    df_translated_combined_new['words'] = df_translated_combined_new.tweet.str.lower().str.split()
    # remove emojis
    df_translated_combined_new["words"] = [[remove_emojis(item) for item in x] for x in df_translated_combined_new["words"]]
    # remove punctuations
    df_translated_combined_new["words"] = [[remove_puncs(item) for item in x] for x in df_translated_combined_new["words"]]
    # remove words from banned list (including user + stopwords)
    df_translated_combined_new["words"] = [[item for item in x if item not in banned] for x in df_translated_combined_new["words"]]
    # remove stopwords
    # df_translated_combined_new["words"] = [[item for item in x if item not in stopwords.words('english')] for x in df_translated_combined_new["words"]]
    # remove words shorter than length 3
    df_translated_combined_new["words"] = [[remove_less_three_char(item) for item in x] for x in df_translated_combined_new["words"]]
    # remove empty strings from list
    df_translated_combined_new["words"] = [[item for item in x if item] for x in df_translated_combined_new["words"]]
    
    return df_translated_combined_new

In [9]:
ha_df = preprocessing(ha_df)
ha_df

,ID,tweet,label,words
0,ha_train_00001,@user Da kudin da Arewa babu wani abin azo aga...,negative,"[kudin, arewa, babu, wani, abin, azo, agani, y..."
1,ha_train_00002,@user Kaga wani Adu ar Banda💔😭 wai a haka Shi ...,negative,"[kaga, wani, adu, banda, wai, haka, shi, shuga..."
2,ha_train_00003,@user Sai haquri fa yan madrid daman kunce cha...,negative,"[sai, haquri, yan, madrid, daman, kunce, champ..."
3,ha_train_00004,@user Hmmm yanzu kai kasan girman allah daxaka...,negative,"[hmmm, yanzu, kai, kasan, girman, allah, daxak..."
4,ha_train_00005,@user @user Wai gwamno nin Nigeria suna afa kw...,negative,"[wai, gwamno, nin, nigeria, suna, afa, kwayoyi]"
...,...,...,...,...
14167,ha_train_14168,"@user Ba wasa a fuskokinsu, may you succeed al...",positive,"[wasa, fuskokinsu, may, succeed, always, guys]"
14168,ha_train_14169,@user Allah yasa muyi kyakkyawan qarshe 🤲🤲,positive,"[allah, yasa, muyi, kyakkyawan, qarshe]"
14169,ha_train_14170,@user Abu nafarko gamawa da duniya lafiya kuma...,positive,"[abu, nafarko, gamawa, duniya, lafiya, kuma, i..."
14170,ha_train_14171,@user @user @user Allah ubangiji yaiwa rayuwar...,positive,"[allah, ubangiji, yaiwa, rayuwarsa, albarka]"


In [13]:
ha_df = ha_df.rename(columns={'tweet': 'text'})
ha_df

,ID,text,label,words
0,ha_train_00001,@user Da kudin da Arewa babu wani abin azo aga...,negative,"[kudin, arewa, babu, wani, abin, azo, agani, y..."
1,ha_train_00002,@user Kaga wani Adu ar Banda💔😭 wai a haka Shi ...,negative,"[kaga, wani, adu, banda, wai, haka, shi, shuga..."
2,ha_train_00003,@user Sai haquri fa yan madrid daman kunce cha...,negative,"[sai, haquri, yan, madrid, daman, kunce, champ..."
3,ha_train_00004,@user Hmmm yanzu kai kasan girman allah daxaka...,negative,"[hmmm, yanzu, kai, kasan, girman, allah, daxak..."
4,ha_train_00005,@user @user Wai gwamno nin Nigeria suna afa kw...,negative,"[wai, gwamno, nin, nigeria, suna, afa, kwayoyi]"
...,...,...,...,...
14167,ha_train_14168,"@user Ba wasa a fuskokinsu, may you succeed al...",positive,"[wasa, fuskokinsu, may, succeed, always, guys]"
14168,ha_train_14169,@user Allah yasa muyi kyakkyawan qarshe 🤲🤲,positive,"[allah, yasa, muyi, kyakkyawan, qarshe]"
14169,ha_train_14170,@user Abu nafarko gamawa da duniya lafiya kuma...,positive,"[abu, nafarko, gamawa, duniya, lafiya, kuma, i..."
14170,ha_train_14171,@user @user @user Allah ubangiji yaiwa rayuwar...,positive,"[allah, ubangiji, yaiwa, rayuwarsa, albarka]"


In [26]:
train_data_path = ha_df_path.rsplit('/', 1)
train_data_path = train_data_path[0] + "/ha/"
train_data_path

'semeval2023-afrisenti-main/data/raw/train/ha/'

In [27]:
## Save to a file
ha_df.to_csv(train_data_path + 'ha_train.tsv', sep='\t', index=False)

## Perform analysis on ha using LinearSVC, Naive Bayes

In [14]:
## Number of unique words in the given data.
# len(set(" ".join(df_am_filtered["text"]).split()))

len(set(" ".join(ha_df["text"]).split()))

39482

In [11]:
VOCAB_SIZE = 35000 


In [31]:
def get_data_and_labels(tsv_path):
    df = pd.read_csv(tsv_path, sep='\t')
    data = df['text'].to_list()
    labels = df['label'].to_list()
    labels = [t.strip().upper() for t in labels]
    return data, np.array(labels)

In [52]:
def identity_tokenizer(text):
    return text

def get_features(corpus, vocab_size):
    vectorizer = CountVectorizer(    ## for multinomial NB
    #vectorizer = TfidfVectorizer(  ## for linear SVC
    ngram_range=(1, 2),
    max_features=vocab_size,
    tokenizer=identity_tokenizer, # already receiving tokenized text from AUtotokenizer
    lowercase=False,
    token_pattern=None
    )
    vectorizer.fit(corpus)
    X = vectorizer.transform(corpus)
    return X, vectorizer

In [57]:
def evaluate_kflod(data, true_labels, tokenizer):
    skf = StratifiedKFold(n_splits=5)   #10
    tokenized_texts_str = [tokenizer.convert_ids_to_tokens(text) for text in tokenizer(data)['input_ids']]
    features,_ = get_features(tokenized_texts_str, VOCAB_SIZE)
    true_test_labels = []
    predicted_test_labels = []
    for train_index, test_index in skf.split(features, true_labels):
        # classifier = LinearSVC(C=0.2,max_iter=1000)  #0.2, C=100,max_iter=10000
        classifier = MultinomialNB()
        train_labels = true_labels[train_index]
        classifier.fit(features[train_index], train_labels)
        train_predictions = classifier.predict(features[train_index])
        test_predictions = classifier.predict(features[test_index])
        test_labels = true_labels[test_index]
        true_test_labels.extend(test_labels)
        predicted_test_labels.extend(test_predictions)
        # train_f1 = f1_score(train_labels, train_predictions, average='micro', labels=['NEGATIVE', 'NEUTRAL','POSITIVE'])
        # test_f1 = f1_score(test_labels, test_predictions, average='micro', labels=['NEGATIVE', 'NEUTRAL','POSITIVE'])
        # print(f"Train F1: {train_f1}, Test F1: {test_f1}")
    return true_test_labels, predicted_test_labels

In [54]:
def evaluate_model_and_dataset(data_set_path):
    data_files = sorted(Path(data_set_path).glob('*.tsv'))
    african_language_model = 'Davlan/afro-xlmr-mini'
    tokenizer = AutoTokenizer.from_pretrained(african_language_model)
    results = []
    
    true_labels_details  = []
    predicted_labels_details  = []
    
    for csv_file in tqdm(data_files, total=len(data_files)):
        language = csv_file.stem.split('_')[0]
        model = "Multinomial Naive Bayes"
        # model = "Linear SVC"

        data, true_labels = get_data_and_labels(csv_file)
        label_set = ['NEGATIVE', 'POSITIVE', 'NEUTRAL']
        true_labels, predicted_labels = evaluate_kflod(data, true_labels, tokenizer)
        accuracy = accuracy_score(true_labels, predicted_labels)
        precision = precision_score(true_labels, predicted_labels, labels=label_set, average=None)
        recall = recall_score(true_labels, predicted_labels, labels=label_set, average=None)
        f1 = f1_score(true_labels, predicted_labels, average='macro')
        
        true_labels_details.append(true_labels)
        predicted_labels_details.append(predicted_labels)
        
        output = {  'model': model,
                    'language': language,
                    'num_examples': len(data),
                    'precision': {label_set[i]: precision[i] for i in range(len(label_set))},
                    'recall': {label_set[i]: recall[i] for i in range(len(label_set))},
                    'f1_score_macro': f1,
                    'accuracy': accuracy
                }
        results.append(output)
    return results, true_labels_details, predicted_labels_details

In [20]:
# train_data_path = ha_df_path.rsplit('/', 1)
# train_data_path = train_data_path[0] + "/ha/"
# print(train_data_path)

semeval2023-afrisenti-main/data/raw/train/ha/


In [58]:
#ha_df_path =  r'semeval2023-afrisenti-main/data/raw/train/ha_train.tsv'

results, tr_labels, pr_labels = evaluate_model_and_dataset(train_data_path)

# df = pd.DataFrame(results)
# df.to_excel('../reports/linear_svm_evaluation.xlsx')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.32s/it]


## Linear SVC 

In [51]:
## linearSVC, cv = 10, C=0.2 
df_results = pd.DataFrame(results)
df_results

,model,language,num_examples,precision,recall,f1_score_macro,accuracy
0,Linear SVC,ha,14172,"{'NEGATIVE': 0.7236648701616855, 'POSITIVE': 0.8634694814134924, 'NEUTRAL': 0.6615491974877878}","{'NEGATIVE': 0.6459654493767767, 'POSITIVE': 0.8028589716236398, 'NEUTRAL': 0.7719869706840391}",0.742396,0.741533


In [45]:
## linearSVC, cv =3, C=0.2
df_results = pd.DataFrame(results)
df_results

,model,language,num_examples,precision,recall,f1_score_macro,accuracy
0,Linear SVC,ha,14172,"{'NEGATIVE': 0.7141802067946824, 'POSITIVE': 0.8527860582435222, 'NEUTRAL': 0.6505479213776308}","{'NEGATIVE': 0.6343756833588454, 'POSITIVE': 0.7934713036057179, 'NEUTRAL': 0.761400651465798}",0.731866,0.731019


In [48]:
## linearSVC, cv =3, C=1
df_results = pd.DataFrame(results)
df_results

,model,language,num_examples,precision,recall,f1_score_macro,accuracy
0,Linear SVC,ha,14172,"{'NEGATIVE': 0.7051551201306275, 'POSITIVE': 0.8503370786516854, 'NEUTRAL': 0.6675252989880405}","{'NEGATIVE': 0.6610540126831401, 'POSITIVE': 0.8073394495412844, 'NEUTRAL': 0.738599348534202}",0.737313,0.736311


In [41]:
## linearSVC, cv =3, C=5
df_results = pd.DataFrame(results)
df_results

,model,language,num_examples,precision,recall,f1_score_macro,accuracy
0,Linear SVC,ha,14172,"{'NEGATIVE': 0.6921842768737108, 'POSITIVE': 0.8362508309328607, 'NEUTRAL': 0.6648413897280967}","{'NEGATIVE': 0.6603979881915591, 'POSITIVE': 0.8052058886281204, 'NEUTRAL': 0.7168159609120521}",0.728735,0.727844


## Multinomial Naive Bayes results

In [56]:
## Multinomial Naive Bayes, cv = 10 
df_results = pd.DataFrame(results)
df_results

,model,language,num_examples,precision,recall,f1_score_macro,accuracy
0,Multinomial Naive Bayes,ha,14172,"{'NEGATIVE': 0.6953996282527881, 'POSITIVE': 0.8612167300380228, 'NEUTRAL': 0.6469964664310954}","{'NEGATIVE': 0.6544937677673299, 'POSITIVE': 0.7732024749306593, 'NEUTRAL': 0.7455211726384365}",0.727313,0.725303


In [59]:
## Multinomial Naive Bayes, cv = 5
df_results = pd.DataFrame(results)
df_results

,model,language,num_examples,precision,recall,f1_score_macro,accuracy
0,Multinomial Naive Bayes,ha,14172,"{'NEGATIVE': 0.6908335257446317, 'POSITIVE': 0.8580246913580247, 'NEUTRAL': 0.6438088470421034}","{'NEGATIVE': 0.654275092936803, 'POSITIVE': 0.7710689140174952, 'NEUTRAL': 0.737785016286645}",0.723961,0.721846
